In [1]:
#importing libraries
import tensorflow as tf
import numpy as np
import tempfile
import time
import os
import zipfile
from tensorflow import keras
import tensorflow_model_optimization as tfmot

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28)
(10000, 28, 28)


In [6]:
#Preprocessing Images

# Normalize the image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

In [7]:
# Defining the model architecture
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),

  keras.layers.Conv2D(filters=24, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),

  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),

  keras.layers.Conv2D(filters=8, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),

  keras.layers.Flatten(),
  keras.layers.Dense(128),
  keras.layers.Dense(64),
  keras.layers.Dense(10)
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 24)        240       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 24)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 12)        2604      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 12)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 8)           8

# A) Before Pruning

In [9]:
# defining the compiler
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
# Calculating Training time
start = time.time() #starting

#Training
model.fit(train_images, train_labels, epochs=10, validation_split=0.1)

end = time.time() #stoping

temp = end - start
print ('Time Taken for Training : %.8f'%temp)

Epoch 1/10
1688/1688 [==============================] - 22s 12ms/step - loss: 0.7298 - accuracy: 0.7261 - val_loss: 0.5689 - val_accuracy: 0.7937
Epoch 2/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.5137 - accuracy: 0.8137 - val_loss: 0.4952 - val_accuracy: 0.8165
Epoch 3/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.4673 - accuracy: 0.8308 - val_loss: 0.4536 - val_accuracy: 0.8318
Epoch 4/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.4400 - accuracy: 0.8419 - val_loss: 0.4166 - val_accuracy: 0.8490
Epoch 5/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.4232 - accuracy: 0.8486 - val_loss: 0.4176 - val_accuracy: 0.8487
Epoch 6/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.4093 - accuracy: 0.8524 - val_loss: 0.4129 - val_accuracy: 0.8503
Epoch 7/10
1688/1688 [==============================] - 20s 12ms/step - loss: 0.4001 - accuracy: 0.8573 - val_loss: 0.4166 -

# Evaluating Model

In [11]:
#calculating accuracy
score = model.evaluate(test_images, test_labels, verbose=0)

# Model Accuracy
print("Accuracy : ",score[1]) 

Accuracy :  0.8535000085830688


In [12]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [19]:
#calculating time taken for predicting one image

image = test_images[5]             # image shape = (28, 28)
image = image.reshape(1, 28, 28)   # image shape = (1, 28, 28)

start = time.time() #starting

#Predicting
predictions = model.predict(image)
print("Prediction : ",class_names[np.argmax(predictions[0])])

end = time.time() #stoping

print("Actual Value: ",class_names[test_labels[5]])

temp=end - start
print ('Time Taken for predicting one data point : %.8f'%temp)

(1, 28, 28)
1/1 [==============================] - 0s 21ms/step
Prediction :  Trouser
Actual Value:  Trouser
Time Taken for predicting one data point : 0.08233523


# Calculating the size of classifier

In [26]:
model.save('model.h5')

In [30]:
print("Size of model : ",os.stat('model.h5').st_size)

Size of model :  223096


# B) After Pruning

In [39]:
batch_size = 128
epochs = 10
validation_split = 0.1  

n_images = train_images.shape[0] * (1 - validation_split) #calculating number of images in training set after subtracting validation set
last_step = np.ceil(n_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_para = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=last_step)
}

pruning_model = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_para)

In [40]:
#defining the compiler
pruning_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#adding callbacks
callback = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=tempfile.mkdtemp()),
]

In [41]:
# Calculating Training time
start = time.time() #starting

#Training
pruning_model.fit(train_images, train_labels,batch_size = batch_size, epochs=epochs, validation_split=validation_split,callbacks=callback)

end = time.time() #stoping

temp = end - start
print ('Time Taken for Training : %.8f'%temp)

Epoch 1/10
422/422 [==============================] - 14s 26ms/step - loss: 0.4384 - accuracy: 0.8377 - val_loss: 0.4353 - val_accuracy: 0.8372
Epoch 2/10
422/422 [==============================] - 10s 24ms/step - loss: 0.4156 - accuracy: 0.8460 - val_loss: 0.4237 - val_accuracy: 0.8430
Epoch 3/10
422/422 [==============================] - 11s 25ms/step - loss: 0.4045 - accuracy: 0.8514 - val_loss: 0.4209 - val_accuracy: 0.8432
Epoch 4/10
422/422 [==============================] - 10s 24ms/step - loss: 0.4010 - accuracy: 0.8528 - val_loss: 0.4266 - val_accuracy: 0.8423
Epoch 5/10
422/422 [==============================] - 10s 25ms/step - loss: 0.4017 - accuracy: 0.8525 - val_loss: 0.4364 - val_accuracy: 0.8377
Epoch 6/10
422/422 [==============================] - 10s 24ms/step - loss: 0.4090 - accuracy: 0.8486 - val_loss: 0.4316 - val_accuracy: 0.8438
Epoch 7/10
422/422 [==============================] - 10s 24ms/step - loss: 0.4121 - accuracy: 0.8488 - val_loss: 0.4309 - val_accuracy:

# Evaluating the model

In [46]:
score = pruning_model.evaluate(test_images, test_labels, verbose=0)
print('Pruned Test loss:', score[0]) 
print('Pruned Test accuracy:', score[1])

Pruned Test loss: 0.4535457491874695
Pruned Test accuracy: 0.8378999829292297


In [48]:
#calculating time taken for predicting one image

image = test_images[5]
image = image.reshape(1, 28, 28)

start = time.time() #starting

#Predicting
predictions = pruning_model.predict(image)
print("Prediction : ",class_names[np.argmax(predictions[0])])

end = time.time() #stoping

print("Actual Value: ",class_names[test_labels[5]])

temp=end - start
print ('Time Taken for predicting one data point : %.8f'%temp)

1/1 [==============================] - 0s 171ms/step
Prediction :  Trouser
Actual Value:  Trouser
Time Taken for predicting one data point : 0.24356222


# Calculating the size of classifier

In [49]:
#performing compression on the model
final_model = tfmot.sparsity.keras.strip_pruning(pruning_model)

In [50]:
final_model.save('model2.h5') 

print("Size of model : ",os.stat('model2.h5').st_size)

Size of model :  88432
